# Importing Superconducting Gravimeter (SG) data from IGETS
Version 2.0  - April 7, 2025  
Created by Chris Linick (Jackson School of Geosciences, UT Austin; clinick@utexas.edu)  
Modified by Bryant Chow (University of Alaska Fairbanks; UAF) for UAF GEOS419 - Solid Earth Geophysics  

- This notebook helps you connect to the IGETS database, browse its contents, download gravity data files, and then make plots. 
- Note that Level 3 files, and possibly level 2, are probably most desirable. Level 1 data requires intensive cleaning.
- [What do the levels mean?](https://isdc.gfz.de/fileadmin/isdc/docs/EOSTproducts_v2.pdf)



In [ ]:
import os
import paramiko  # for connecting to IGETS server via sftp
import numpy as np
import traceback
import matplotlib.pyplot as plt 
from datetime import datetime

## Step 1: Connect to Server
1) Sign up for an IGETS account (confirmation is immediate): https://isdc.gfz.de/igets-data-base/
2) Wait to receive confirmation email that will provide you your `username`, which is a variation of your input email.
3) Change the 3 variables (`username`, `password`, `save_location_local`) in the code box below to match your credentials.
4) If authentication fails, check your username and password, and your internet connection.
5) If succesful, a list of stations in the IGETS database should be printed out.

In [ ]:
username = ""  # <- Input your username
password = ""  # <- Input your password
save_location_local = "./" # <- Determine where files will be saved

In [ ]:
# Establish connection to the server, do not change anything in this code block
server = "igetsftp.gfz.de"
try:
    # Establish SSH client
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    # Connect to server
    ssh.connect(server, username=username, password=password)
    
    # Open SFTP session
    sftp = ssh.open_sftp()
    
    # List contents of the top directory
    contents = sftp.listdir("/")
    
    print("Contents of the top directory:")
    print(contents)
except Exception as e:
    traceback.print_exc()
    print(f"Error: {e}")

## Step 2: Browse for the SG data you want.
- In this example, let's aim for level 3 data from Onsala Space Observatory in Sweden, for the month of July in 2021.
- We'll step down one directory at a time until we find what we're looking for.
- The directory structure is: `station` > `instrument` > `processing_level` > `year` > `month`

In [ ]:
# The station we want is Onsala Space Observatory (change line below if you want a different station)
loc = "Onsala"
contents = sftp.listdir(f"/{loc}")
print(contents)

In [ ]:
# There is only one instrument at Onsala -- an observatory SG with serial number 54. So we check contents of this folder.
inst = "os054"
contents = sftp.listdir(f"/{loc}/{inst}")
print(contents)

In [ ]:
# Okay so they have Level 1, 2, and 3 data available. We want level 3.
lev = "Level3"
contents = sftp.listdir(f"/{loc}/{inst}/{lev}")
print(contents)

In [ ]:
# Let's see which months are in the 2021 directory.
yr = "2021"
contents = sftp.listdir(f"/{loc}/{inst}/{lev}/{yr}")
print(contents)

In [ ]:
# The July file is 'IGETS-SG-RESMIN-os054-202107r2.ggp'. That wasn't so bad.
fname = "IGETS-SG-RESMIN-os054-202107r2.ggp"
fpath = f"/{loc}/{inst}/{lev}/{yr}/{fname}"
print(fpath)

## Step 3: Download file and Close SFTP Session

In [ ]:
# Use SFTP to get the file we want
fout = os.path.join(save_location_local, fname)
sftp.get(fpath, fout)

# Close the SFTP session and SSH connection
sftp.close()
ssh.close()

## Step 4: Import and Plot File
- Run the code below to import the .ggp file you downloaded.
- Please be aware that the import function might only read a portion of the file, since I have not rigorously designed this according to GGP file specs [discussed here](https://gfzpublic.gfz-potsdam.de/rest/items/item_5003806_1/component/file_5003807/content)

In [ ]:
# A basic GGP file import function
def parse_igets_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Find the line containing variable names
    for i, line in enumerate(lines):
        if line.startswith("yyyymmdd hhmmss"):
            header_line = line.strip()
            break
    else:
        raise ValueError("Header with column names not found")

    # Extract column names, skipping yyyymmdd and hhmmss
    col_names = header_line.split()[2:]  

    # Find the start of the data section
    for i, line in enumerate(lines):
        if line.strip() == "77777777":
            data_start_idx = i + 1
            break
    else:
        raise ValueError("Data start marker '77777777' not found")

    # Find the first occurrence of 88888888 or 99999999 to stop reading
    for j in range(data_start_idx, len(lines)):
        if lines[j].strip() in ("88888888", "99999999"):
            data_end_idx = j
            break
    else:
        data_end_idx = len(lines)  # Read until the end if no termination marker is found

    # Read raw data lines
    raw_data = lines[data_start_idx:data_end_idx]

    # Initialize lists to store parsed data
    datetime_list = []
    data_values = []

    for line in raw_data:
        parts = line.split()
        
        # Parse date
        yyyymmdd = parts[0]
        hhmmss = parts[1].zfill(6)  # Ensure hhmmss is always 6 characters (zero-padded if needed)

        # Convert to datetime
        dt_str = f"{yyyymmdd} {hhmmss[:2]}:{hhmmss[2:4]}:{hhmmss[4:]}"
        dt_obj = datetime.strptime(dt_str, "%Y%m%d %H:%M:%S")
        datetime_list.append(dt_obj)

        # Store remaining values as floats
        data_values.append([float(x) for x in parts[2:]])

    # Convert to NumPy arrays
    datetime_array = np.array(datetime_list, dtype="datetime64[s]")
    data_array = np.array(data_values)

    # Convert to dictionary
    data_dict = {"datetime": datetime_array}
    for i, name in enumerate(col_names):
        data_dict[name] = data_array[:, i]

    return data_dict

# Now parse the file we just downloaded
sgdata = parse_igets_file(fout)

## Step 5: Plot the SG Residual (or another quantity of your choosing)
You can change the quantity plotted by setting `key_to_plot` below. The quantities that you can plot are listed below:
- `res_fil`: residual in nm/s^2 (after removal of tides, atmospheric loading, and earth rotation effects, with gaps/earthquakes/artifacts filled with synthetic data)
- `res_nofil`: residual, unfilled, in nm/s^2
- `tides`: solid Earth and ocean tide model (nm/s^2)
- `rotation`: Earth rotation effects model (polar motion + length of day) in nm/s^2
- `atm_load`: atmospheric loading in nm/s^2
- `drift`: secular drift of instrument in nm/s^2
- `g_fil`: original gravity (filled)
- `p_fil`: original baro pressure (filled) in mbar/hPZ


You can adjust the time to be plotted by changing `t1` and `t2`

In [ ]:
# Here are the keys (columns) of the imported sg data that you can plot
print(list(sgdata.keys()))

key_to_plot = "res_fil"

key_to_label_dict = {
    "res_fil": "Filled Gravity Residual",
    "res_nofil": "Non-filled Gravity Residual",
    "tides": "Solid Earth and Ocean Tides",
    "rotation": "Rotation", 
    "atm_load": "Atmospheric Loading",
    "drift": "Secular Drift",
    "g_fil": "Filled Gravity",
    "p_fil": "Filled Barometric Pressure",
}

# User-defined time range (modify as needed)
t1 = "July 1, 2021 12:00:00"
t2 = "July 10, 2021 13:30:00"

# Convert to NumPy datetime64 format
t1 = np.datetime64(datetime.strptime(t1, "%B %d, %Y %H:%M:%S"))
t2 = np.datetime64(datetime.strptime(t2, "%B %d, %Y %H:%M:%S"))

# Filter data within the specified time range
mask = (sgdata["datetime"] >= t1) & (sgdata["datetime"] <= t2)
time_subset = sgdata["datetime"][mask]
res_fil_subset = sgdata[key_to_plot][mask]

# Plot the figure
plt.figure(figsize=(10, 5))
plt.plot(time_subset, res_fil_subset, color="b")
plt.xlabel("Time [UTC]")
if key_to_plot == "p_fil":
    units = "mbar/hPZ"
else:
    units = "nm/s$^2$"
plt.ylabel(f"{key_to_label_dict[key_to_plot]} [{units}]")
plt.title(f"{lev} Data at {loc}")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()